In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import confusion_matrix, SCORERS, classification_report
import numpy as np

In [ ]:
featuresCD = pd.read_csv("/content/drive/MyDrive/NYC DSA Capstone Project/Data/featuresCD.csv", low_memory = False)
featuresCD.drop(["Unnamed: 0"], axis = 1, inplace = True)
featuresCD

,loan_status_trim,state_income,annual_inc_joint,indiv_app,dti_joint,days_from_1st_cr_line,emp_length,fico_range_high,fico_range_low,funded_amnt,home_ownership,inq_fi,inq_last_12m,installment,int_rate,loan_amnt,open_acc,pub_rec,pub_rec_bankruptcies,for_debt_consol,total_acc,revol_bal,revol_util,delinq_amnt,delinq_2yrs,mths_since_last_delinq,mths_since_last_major_derog,grade
0,0,2,55000.0,1,5.91,4505.0,10,679.0,675.0,3600.0,1,3.0,4.0,123.03,13.99,3600.0,7.0,0.0,0.0,1,13.0,2765.0,29.7,0.0,0.0,30.0,30.0,4
1,0,1,65000.0,1,16.06,5844.0,10,719.0,715.0,24700.0,1,0.0,6.0,820.28,11.99,24700.0,22.0,0.0,0.0,0,38.0,21470.0,19.2,0.0,1.0,6.0,0.0,4
2,0,2,34000.0,1,10.20,10288.0,4,694.0,690.0,11950.0,0,0.0,0.0,405.18,13.44,11950.0,5.0,0.0,0.0,1,6.0,8822.0,68.4,0.0,0.0,0.0,0.0,4
3,0,2,64000.0,1,34.95,7122.0,3,704.0,700.0,1400.0,1,0.0,0.0,47.10,12.88,1400.0,17.0,0.0,0.0,0,24.0,37828.0,67.2,0.0,0.0,0.0,0.0,4
4,0,2,112000.0,1,8.68,8065.0,10,804.0,800.0,18000.0,1,0.0,1.0,400.31,11.99,18000.0,17.0,0.0,0.0,1,27.0,10711.0,15.5,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583975,0,2,119000.0,1,32.73,8827.0,10,689.0,685.0,23800.0,1,4.0,10.0,559.85,14.49,23800.0,13.0,0.0,0.0,1,29.0,107747.0,89.5,0.0,0.0,0.0,0.0,4
583976,0,3,180792.0,1,22.03,5328.0,9,709.0,705.0,29400.0,1,3.0,2.0,683.94,13.99,29400.0,16.0,0.0,0.0,1,32.0,77480.0,85.2,0.0,0.0,0.0,0.0,4
583977,1,2,157000.0,1,10.34,1949.0,3,739.0,735.0,32000.0,1,2.0,0.0,752.74,14.49,32000.0,14.0,0.0,0.0,0,18.0,111598.0,27.4,0.0,0.0,0.0,0.0,4
583978,0,2,150000.0,1,12.25,7001.0,10,669.0,665.0,16000.0,0,1.0,1.0,362.34,12.79,16000.0,12.0,4.0,3.0,0,28.0,7700.0,55.0,0.0,0.0,68.0,68.0,4


In [ ]:
featuresCD.columns

Index(['loan_status_trim', 'state_income', 'annual_inc_joint', 'indiv_app',
       'dti_joint', 'days_from_1st_cr_line', 'emp_length', 'fico_range_high',
       'fico_range_low', 'funded_amnt', 'home_ownership', 'inq_fi',
       'inq_last_12m', 'installment', 'int_rate', 'loan_amnt', 'open_acc',
       'pub_rec', 'pub_rec_bankruptcies', 'for_debt_consol', 'total_acc',
       'revol_bal', 'revol_util', 'delinq_amnt', 'delinq_2yrs',
       'mths_since_last_delinq', 'mths_since_last_major_derog', 'grade'],
      dtype='object')

In [ ]:
data = featuresCD.iloc[:, 1:]
target = featuresCD.iloc[:, 0]

In [ ]:
data

,annual_inc_joint,indiv_app,dti_joint,days_from_1st_cr_line,emp_length,fico_range_high,fico_range_low
0,55000.0,1,5.91,4505.0,10,679.0,675.0
1,65000.0,1,16.06,5844.0,10,719.0,715.0
2,34000.0,1,10.20,10288.0,4,694.0,690.0
3,64000.0,1,34.95,7122.0,3,704.0,700.0
4,112000.0,1,8.68,8065.0,10,804.0,800.0
...,...,...,...,...,...,...,...
583975,119000.0,1,32.73,8827.0,10,689.0,685.0
583976,180792.0,1,22.03,5328.0,9,709.0,705.0
583977,157000.0,1,10.34,1949.0,3,739.0,735.0
583978,150000.0,1,12.25,7001.0,10,669.0,665.0


In [ ]:
target

0         0
1         0
2         0
3         0
4         0
         ..
583975    0
583976    0
583977    1
583978    0
583979    1
Name: loan_status_trim, Length: 583980, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, stratify = target, shuffle = True)

In [ ]:
%%time

print("-"*60)
tree_model = tree.DecisionTreeClassifier(criterion = "entropy",
                                         class_weight = {0:2e-8, 1:1},
                                         #min_sample 
                                         random_state = 0)

tree_model.fit(X_train, y_train)

print(f"Training score: {tree_model.score(X_train, y_train)}")
print(f"Test score:     {tree_model.score(X_test, y_test)}")

print("-"*60)

print(f"Training Confusion Matrix")
print(confusion_matrix(y_true = y_train,
                       y_pred = tree_model.predict(X_train)))

print("-"*60)

print(f"Testing Confusion Matrix")
print(confusion_matrix(y_true = y_test,
                       y_pred = tree_model.predict(X_test)))

print("-"*60)

------------------------------------------------------------
Training score: 0.9865088334727706
Test score:     0.648115803052616
------------------------------------------------------------
Training Confusion Matrix
[[300323   5515]
 [     0 102948]]
------------------------------------------------------------
Testing Confusion Matrix
[[101958  29115]
 [ 32533  11588]]
------------------------------------------------------------
CPU times: user 42.1 s, sys: 69.7 ms, total: 42.2 s
Wall time: 42.3 s


In [ ]:
pd.DataFrame(list(zip(featuresCD.columns, tree_model.feature_importances_)), columns = ["feature", "importance"]).sort_values(by = "importance", ascending = False)

,feature,importance
13,installment,1.0
1,state_income,0.0
24,delinq_2yrs,0.0
23,delinq_amnt,0.0
22,revol_util,0.0
21,revol_bal,0.0
20,total_acc,0.0
19,for_debt_consol,0.0
18,pub_rec_bankruptcies,0.0
17,pub_rec,0.0


Look at Gradient Boosting again

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, train_size = 0.01, test_size = 0.01, stratify = target, shuffle = True)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import sklearn.model_selection as ms

In [ ]:
param_grid = {
    "learning_rate" : [0.01],
    "max_depth"     : [5, 10, 15],
    "subsample"     : [0.9],
    "n_estimators"  : [300,]
}

In [ ]:
%%time
gbm = GradientBoostingClassifier()
grid_search_gbm = ms.GridSearchCV(gbm, param_grid, scoring = 'recall', cv = 3, n_jobs = -1)
grid_search_gbm.fit(X_train, y_train)

CPU times: user 713 ms, sys: 0 ns, total: 713 ms
Wall time: 4.91 s


In [ ]:
grid_search_gbm.best_params_

{'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.9}

In [ ]:
grid_search_gbm.best_score_

0.0

In [ ]:
print("-"*60)

print(f"Training Confusion Matrix")
print(confusion_matrix(y_true = y_train,
                       y_pred = grid_search_gbm.best_estimator_.predict(X_train)))

print("-"*60)

print(f"Testing Confusion Matrix")
print(confusion_matrix(y_true = y_test,
                       y_pred = grid_search_gbm.best_estimator_.predict(X_test)))

print("-"*60)

------------------------------------------------------------
Training Confusion Matrix
[[43683     8]
 [14608    99]]
------------------------------------------------------------
Testing Confusion Matrix
[[43656    35]
 [14657    50]]
------------------------------------------------------------


In [ ]:
grid_search_gbm.cv_results_

{'mean_fit_time': array([0.86347795, 0.89097754, 0.80033247]),
 'mean_score_time': array([0.02680675, 0.02546533, 0.02266534]),
 'mean_test_score': array([0., 0., 0.]),
 'param_learning_rate': masked_array(data=[0.01, 0.01, 0.01],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[5, 10, 15],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[300, 300, 300],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_subsample': masked_array(data=[0.9, 0.9, 0.9],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'learning_rate': 0.01,
   'max_depth': 5,
   'n_estimators': 300,
   'subsample': 0.9},
  {'learning_rate': 0.01,
   'max_depth': 10,
   'n_estimators': 300,
   'subsample': 0.9},
  {'learning_rate': 0.01,
   'max_de

In [ ]:
# Can't use class_weight for Gradient Boosting

class_weight = {0:1, 
                1: len(featuresCD[featuresCD["loan_status_trim"] == 0]) / len(featuresCD[featuresCD["loan_status_trim"] == 1])
                }

In [ ]:
from sklearn.svm import SVC

In [ ]:
%%time 

X_train, X_test, y_train, y_test = train_test_split(data, target, train_size = 0.01, test_size = 0.01, stratify = target, shuffle = True)

svm = SVC()

weights = np.linspace(2e-16, 3, 11)

param_grid_svm = {
    "kernel": ["sigmoid"],
    "class_weight" : [{0:x, 1:3-x} for x in weights]
    }

grid_search_svm = ms.GridSearchCV(svm, 
                                  param_grid_svm, 
                                  scoring = "f1_macro", 
                                  cv = 2, 
                                  n_jobs = -1)

grid_search_svm.fit(X_train, y_train)
print(grid_search_svm.best_estimator_.score(X_train, y_train))
print(grid_search_svm.best_estimator_.score(X_test, y_test))
print(confusion_matrix(y_true = y_train, y_pred = grid_search_svm.best_estimator_.predict(X_train)))
print(confusion_matrix(y_true = y_test, y_pred = grid_search_svm.best_estimator_.predict(X_test)))
print(classification_report(y_test, y_pred = grid_search_svm.best_estimator_.predict(X_test)))
print(grid_search_svm.best_params_)

0.5992464463092996
0.5934931506849315
[[2988 1381]
 [ 959  511]]
[[2974 1395]
 [ 979  492]]
              precision    recall  f1-score   support

           0       0.75      0.68      0.71      4369
           1       0.26      0.33      0.29      1471

    accuracy                           0.59      5840
   macro avg       0.51      0.51      0.50      5840
weighted avg       0.63      0.59      0.61      5840

{'class_weight': {0: 1.2000000000000002, 1: 1.7999999999999998}, 'kernel': 'sigmoid'}
CPU times: user 7.04 s, sys: 26.1 ms, total: 7.07 s
Wall time: 17.2 s


In [ ]:
%%time 

X_train, X_test, y_train, y_test = train_test_split(data, target, train_size = 0.01, test_size = 0.01, stratify = target, shuffle = True)

svm = SVC(class_weight = {0:1.2, 1:1.8}, kernel = "sigmoid", C = 33)

param_grid_svm = {
    "C":[33]
    }

grid_search_svm = ms.GridSearchCV(svm, 
                                  param_grid_svm, 
                                  scoring = "f1_macro", 
                                  cv = 2, 
                                  n_jobs = -1)

grid_search_svm.fit(X_train, y_train)
print(grid_search_svm.best_estimator_.score(X_train, y_train))
print(grid_search_svm.best_estimator_.score(X_test, y_test))
print(confusion_matrix(y_true = y_train, y_pred = grid_search_svm.best_estimator_.predict(X_train)))
print(confusion_matrix(y_true = y_test, y_pred = grid_search_svm.best_estimator_.predict(X_test)))
print(classification_report(y_test, y_pred = grid_search_svm.best_estimator_.predict(X_test)))
print(grid_search_svm.best_params_)
print(grid_search_svm.cv_results_)

0.5867443055317692
0.5885273972602739
[[2917 1452]
 [ 961  509]]
[[2936 1433]
 [ 970  501]]
              precision    recall  f1-score   support

           0       0.75      0.67      0.71      4369
           1       0.26      0.34      0.29      1471

    accuracy                           0.59      5840
   macro avg       0.51      0.51      0.50      5840
weighted avg       0.63      0.59      0.60      5840

{'C': 33}
{'mean_fit_time': array([0.48763037]), 'std_fit_time': array([0.00810599]), 'mean_score_time': array([0.3273046]), 'std_score_time': array([0.00724983]), 'param_C': masked_array(data=[33],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'C': 33}], 'split0_test_score': array([0.51575292]), 'split1_test_score': array([0.49640837]), 'mean_test_score': array([0.50608064]), 'std_test_score': array([0.00967227]), 'rank_test_score': array([1], dtype=int32)}
CPU times: user 5.92 s, sys: 12 ms, total: 5.94 s
Wall time: 6.77 s


In [ ]:
import imblearn
from imblearn.over_sampling import SVMSMOTE
from imblearn.pipeline import make_pipeline

In [ ]:
%%time 

X_train, X_test, y_train, y_test = train_test_split(data, target, train_size = 0.01, test_size = 0.01, stratify = target, shuffle = True)

svm = SVC(#class_weight = {0:1.2, 1:1.8},
          kernel = "sigmoid", 
          C = 33)
          #tol = 2e-16, 
          #shrinking = False
          #break_ties = True)

pipe = make_pipeline(
    SVMSMOTE(svm_estimator = svm,
             sampling_strategy = {0:4369, 1:4369},
             n_jobs = -1,
             m_neighbors = 1,
             k_neighbors = 2),
    svm
)

param_grid_svm = {
    
    }

pipe_grid_search_svm = ms.GridSearchCV(pipe, 
                                  param_grid_svm, 
                                  scoring = "f1_macro", 
                                  cv = 2, 
                                  n_jobs = -1)

pipe_grid_search_svm.fit(X_train, y_train)
print(pipe_grid_search_svm.best_estimator_.score(X_train, y_train))
print(pipe_grid_search_svm.best_estimator_.score(X_test, y_test))
print(confusion_matrix(y_true = y_train, y_pred = pipe_grid_search_svm.best_estimator_.predict(X_train)))
print(confusion_matrix(y_true = y_test, y_pred = pipe_grid_search_svm.best_estimator_.predict(X_test)))
print(classification_report(y_test, y_pred = pipe_grid_search_svm.best_estimator_.predict(X_test)))
#print(pipe_grid_search_svm.best_params_)
print(pipe_grid_search_svm.cv_results_)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

0.5077924302106525
0.5061643835616438
[[2134 2235]
 [ 639  831]]
[[2126 2243]
 [ 641  830]]
              precision    recall  f1-score   support

           0       0.77      0.49      0.60      4369
           1       0.27      0.56      0.37      1471

    accuracy                           0.51      5840
   macro avg       0.52      0.53      0.48      5840
weighted avg       0.64      0.51      0.54      5840

{'mean_fit_time': array([9.03240633]), 'std_fit_time': array([0.39526963]), 'mean_score_time': array([1.66108167]), 'std_score_time': array([0.05238354]), 'params': [{}], 'split0_test_score': array([0.48706566]), 'split1_test_score': array([0.50899916]), 'mean_test_score': array([0.49803241]), 'std_test_score': array([0.01096675]), 'rank_test_score': array([1], dtype=int32)}
CPU times: user 13.8 s, sys: 44 ms, total: 13.8 s
Wall time: 25.3 s


In [ ]:
print("Imbalanced-Learn", imblearn.__version__)

Imbalanced-Learn 0.4.3


In [ ]:
sorted(SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [ ]:
grid_search_svm.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__C', 'estimator__break_ties', 'estimator__cache_size', 'estimator__class_weight', 'estimator__coef0', 'estimator__decision_function_shape', 'estimator__degree', 'estimator__gamma', 'estimator__kernel', 'estimator__max_iter', 'estimator__probability', 'estimator__random_state', 'estimator__shrinking', 'estimator__tol', 'estimator__verbose', 'estimator', 'iid', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 'return_train_score', 'scoring', 'verbose'])

In [ ]:
dir(grid_search_svm)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_cache',
 '_abc_negative_cache',
 '_abc_negative_cache_version',
 '_abc_registry',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',

In [ ]:
grid_search_svm.cv_results_

{'mean_fit_time': array([0.30307233]),
 'mean_score_time': array([0.20949078]),
 'mean_test_score': array([0.12224449]),
 'param_kernel': masked_array(data=['sigmoid'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'sigmoid'}],
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([0.24448898]),
 'split1_test_score': array([0.]),
 'std_fit_time': array([0.04064405]),
 'std_score_time': array([0.00484037]),
 'std_test_score': array([0.12224449])}